<a href="https://colab.research.google.com/github/redadmiral/python-for-journalists/blob/main/Scraper_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping mit Selenium

Wir kennen zwei Möglichkeiten, Informationen aus dem Internet zu scrapen.

Manche Internetseiten hängen sehr stark von Javascript (JS) ab, einer Programmiersprache, mit der man unter anderem Webseiten bauen kann. Manchmal wird die Website nicht fertig ausgeliefert, sondern der Server schickt nur den JS-Code, der dann im Browser ausgeführt wird. 

Der JS-Code lädt dann Informationen nach und baut das HTML-Dokument im Browser. Das Problem dabei ist: Wenn wir einen GET-Request machen, bekommen wir nur den JS-Code statt dem eigentlichen Seiteninhalt:

In [1]:
import requests

response = requests.get("https://k9data.com/verify.asp?breed=1")

In [2]:
response.content.decode("utf-8")

'<html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><title>Error</title><LINK rel="stylesheet" type="text/css" href="ipedigree.css"></head><body><p><table border=0 bgcolor="#000099" cellspacing=0 cellpadding=10 width="100%" ID="Table1"><tr><td valign=top width=125><center><img src="gr_icon.jpg" width=125 height=115></center></td><td valign=center><strong><a STYLE="text-decoration: none" href="default.asp"><font color="#ffffff" size="6" face="verdana">K9data.com</a></font></strong></td><td align=center><font color="#ffffff" size="4" face="verdana"><p><strong>Golden Retriever</strong></p><p><font color="#ffffff" size="2" face="verdana">Looking for a Golden Retriever puppy? <a href="https://www.grca.org/find-a-golden/" style="color: #33CCFF">Click here</a>.</font><p></font></td></tr></table></p><script src="//pmetrics.performancing.com/js" type="text/javascript"></script><script type="text/javascript">try{ clicky.init(17263); }catch(e){}</script><noscript>

Um diese Seiten scrapen zu können, brauchen wir eine Möglichkeit, um den JS-Code auszuführen und die Seite zu rendern.

## Das Notebook vorbereiten

Dafür gibt es die Möglichkeit einen Browser wie Chrome oder Firefox mit Python auszuführen und fernzusteuern. Dafür brauchen wir einerseits den Browser, wir verwenden hier Chrome, und den chromdriver, der eine Schnittstelle zwischen Python und Chrome bereitstellt um den Browser steuern zu können.

Außerdem benutzen wir Selenium, was eine Bibliothek in Python ist, um wiederum mit dem Chromedriver arbeiten zu können.

In der folgenden Box installieren wir zuerst Selenium, laden uns die aktuelle Version von Chrome herunter, installieren diese in unserem Notebook und geben uns die aktuelle Version aus. 

In [ ]:
!pip install selenium
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb

Als nächstes müssen wir die korrekte Version des chromedrivers herunterladen. Diese ist immer abhängig von der verwendeten Version von Chrome. Die installierte Version von Chrome könnt ihr über diesen Befehl ausgeben:

In [4]:
!google-chrome --version

Google Chrome 110.0.5481.177 


Jetzt wo ich dieses Notebook schreibe, ist die aktuelle Version 110.0.5481.177. Wenn ihr das Notebook ausführt, ist sie wahrscheinlich neuer. 

Auf der Seite [chromedriver.chromium.org](https://chromedriver.chromium.org/downloads) können wir die Version auswählen, die wir brauchen. Wichtig ist dafür die erste Zahl, in meinem Fall also die 110. Mit rechtsklick könnt ihr den Link kopieren und fügt ihn in der nächsten Zeile hinter `!wget` ein. Der Link sollte so ähnlich aussehen, wie der Link unten, nur die Versionsnummer sollte eine andere sein.

In [5]:
!wget https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip

--2023-03-01 10:34:42--  https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7396711 (7.1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   7.05M  --.-KB/s    in 0.04s   

2023-03-01 10:34:42 (163 MB/s) - ‘chromedriver_linux64.zip’ saved [7396711/7396711]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


Jetzt haben wir alles installiert was wir brauchen.

Als nächstes richten wir den Browser ein. Wir müssen ihm einige Optionen mitgeben, damit er in einem Notebook funktioniert. Das machen wir über die `options.add_argument()`-Methode.

Die beiden Optionen, die wir mitgeben sind:

- `--headless`: Der Browser wird ausgeführt, ohne dass ein sichtbares Fenster aufgeht. Das müssen wir machen, weil das Notebook kein Browserfenster darstellen kann.
- `--no-sandbox`: Dieses Flag brauchen wir, weil wir Chrome mit Admin-Rechten ausführen. Mit diesem Flag können Websites direkt auf das Notebook zugreifen, das ist hier nicht schlimm, weil die Umgebung in der das Notebook läuft ohnehin gelöscht wird, wenn wir sie nicht mehr benutzen. Benutzt dieses Flag nicht, wenn ihr es lokal auf eurem Rechner ausführt!

Anschließend starten wir Chrome und übergeben unsere Optionen und den Pfad zum chromedriver. Wir speichern die aktuelle Chrome-Session in einer Variable `browser`.

In [10]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')

browser = webdriver.Chrome(executable_path='chromedriver',options=options)

Jetzt können wir anfangen mit dem Browser Seiten aufzurufen. Als erstes gehen wir auf die Startseite von der Hundedatenbank [k9data.com](http://k9data.com).

Mit dem `title`-Attribut können wir uns ausgeben lassen, was gerade in der Kopfzeile des Browsers steht. Damit können wir kurz kontrollieren, ob wir auch auf der Seite sind auf die wir wollen.

In [14]:
browser.get("https://k9data.com/")
browser.title

'K9DATA.COM Home Page'

Weil wir das Browserfenster nicht sehen können, ist es ein bisschen awkward auf der Seite zu navigieren. Immerhin können wir mit der `.save_screenshot()`-Methode Screenshots speichern, wie der Browser gerade aussieht:

In [ ]:
browser.save_screenshot("startseite.png")

Der Screenshot `startseite.png` ist jetzt links in unserem Environment gespeichert. Wenn wir darauf klicken, sehen wir das wir tatsächlich auf der Startseite sind.

Als nächstes müssen wir einen Hundenamen in das Suchfeld eingeben. Dafür wählen wir zuerst das Suchfeld aus. Das geht mit der `find_element()`-Methode. [In der Dokumentation](https://selenium-python.readthedocs.io/locating-elements.html) sind verschiedene Möglichkeiten aufgeführt, wie wir das Element auswählen können:

```python
find_element(By.ID, "id")
find_element(By.NAME, "name")
find_element(By.XPATH, "xpath")
find_element(By.LINK_TEXT, "link text")
find_element(By.PARTIAL_LINK_TEXT, "partial link text")
find_element(By.TAG_NAME, "tag name")
find_element(By.CLASS_NAME, "class name")
find_element(By.CSS_SELECTOR, "css selector")
```

In den Entwicklertools im Browser sehen wir, dass das Element die ID `name` hat, was meistens die einfachste Art ist solch ein Element zu wählen. Die ausgewählte Suchleiste können wir in einer eigenen Variable speichern,, die wir `input_field` nennen.

In [15]:
from selenium.webdriver.common.by import By

input_field = browser.find_element(By.ID, "name")
input_field

<selenium.webdriver.remote.webelement.WebElement (session="03f8a87a5e2452014a4230afd34a0f35", element="317569d6-8d31-4da5-b5f1-8f26677d1fbb")>

Mit der `send_keys()`-Methode können wir etwas in die Suchleiste eintippen. Hier suchen wir nach allen Hunden, die Bello heißen:

In [16]:
input_field.send_keys("Bello")

Und wenn wir jetzt einen Screenshot von der Seite machen, sehen wir dass in der Suchleiste jetzt tatsächlich der Name "Bello" steht.

In [17]:
browser.save_screenshot("suchleiste_ausgefuellt.png")

True

Um auf den Search-Button zu klicken, müssen wir ihn wieder zuerst auswählen. Leider hat der Search-Button keine ID. Deshalb müssen wir den XPATH nutzen. Der XPATH ist eine Art den Weg in einem HTML-Dokument zu einem gewissen Element zu beschreiben. 

Zum Glück müssen wir ihn uns nicht selbst suchen, sondern können ihn uns direkt aus den Entwicklertools ausgeben lassen, indem wir rechts auf das Element im Elements-Tab klicken und "Copy > Copy full XPath" auswählen.

Diesen übergeben wir wieder als Parameter an die `find_element()`-Methode.

In [18]:
search_button = browser.find_element(By.XPATH, "/html/body/form/table/tbody/tr[1]/td/input[2]")

Jetzt müssen wir nur noch auf den Button klicken: Das machen wir mit der `click()`-Methode:

In [19]:
search_button.click()

Wenn wir das in unserem Browser machen, müssen wir kurz warten bis die neue Seite geladen hat. Nachdem wir hier den Browser nicht sehen können, müssen wir dem Browser explizit sagen, dass er warten muss bis die neue Seite geladen hat.

Damit der Browser weiß, dass eine Seite fertig geladen ist, müssen wir ihm ein Element verraten, dass auf der neu geladenen Seite auftaucht. Eine Möglichkeit ist der Link "Click here to return to the home page" auf der Ergebnis-Seite. 

Dafür gibt es die `WebDriverWait()`-Klasse. Wir übergeben ihr zuerst unseren Browser und eine Zeit in Sekunden, die er maximal warten soll. Mit der `until()`-Methode können wir dann verschiedene Szenarien übergeben, ab wann die Seite geladen ist. Für unseren Case passt `.presence_of_element_located()` aus dem `expected_conditions` am besten. 

In der [Dokumentation](https://selenium-python.readthedocs.io/waits.html#explicit-waits) finden wir noch eine ganze Reihe anderer Möglichkeiten, wie wir den Browser auf bestimmte Elemente warten lassen können.

Das Element das erscheinen muss, übergeben wir genauso wie oben als wir die Suchleiste und den Search-Button gesucht haben, nur dieses Mal mit dem Text, der im Link steht:

In [20]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.LINK_TEXT, "Click here to return to the home page")))

<selenium.webdriver.remote.webelement.WebElement (session="03f8a87a5e2452014a4230afd34a0f35", element="5a6f486e-0b78-4384-85d7-57a7b98c8654")>

Wenn diese Methode durchgeladen ist, können wir uns einen Screenshot ansehen, um zu sehen, wie die Ergebnisseite aussieht und/oder uns den Title der Homepage ansehen.

In [21]:
browser.save_screenshot("neue_seite.png")

browser.title

'Resolve name: Bello'

Über das `.page-source`-Attribut können wir uns den HTML-Code der gerenderten Seite ausgeben lassen:

In [22]:
browser.page_source

'<html><head><meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><title>Resolve name: Bello</title></head><body><p><table border="0" bgcolor="#000099" cellspacing="0" cellpadding="10" width="100%" id="Table1"><tbody><tr><td valign="top" width="125"><center><img src="gr_icon.jpg" width="125" height="115"></center></td><td valign="center"><strong><a style="text-decoration: none" href="default.asp"><font color="#ffffff" size="6" face="verdana">K9data.com</font></a></strong></td><td align="center"><font color="#ffffff" size="4" face="verdana"><p><strong>Golden Retriever</strong></p><p><font color="#ffffff" size="2" face="verdana">Looking for a Golden Retriever puppy? <a href="https://www.grca.org/find-a-golden/" style="color: #33CCFF">Click here</a>.</font></p></font><p><font color="#ffffff" size="4" face="verdana"></font></p></td></tr></tbody></table></p><script src="//pmetrics.performancing.com/js" type="text/javascript"></script><script type="text/javascript">try{ cl

Diesen können wir nun wie gewohnt an Beautiful Soup übergeben...

In [23]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(browser.page_source)

...und uns zum Beispiel alle Links daraus ziehen:

In [24]:
soup.find_all("a")

[<a href="default.asp" style="text-decoration: none"><font color="#ffffff" face="verdana" size="6">K9data.com</font></a>,
 <a href="https://www.grca.org/find-a-golden/" style="color: #33CCFF">Click here</a>,
 <a href="pedigree.asp?ID=1249652">Abbybell Odin Jack (2022)</a>,
 <a href="pedigree.asp?ID=989223">Alphinbrook Sleigh Bells of Canina</a>,
 <a href="pedigree.asp?ID=340870">Amanda Bell Of Basil (1991)</a>,
 <a href="pedigree.asp?ID=1079288">Amore Bello Keeper of Our Hearts (2020)</a>,
 <a href="pedigree.asp?ID=524118">Angelic Magnolia Maggie Abello (2010)</a>,
 <a href="pedigree.asp?ID=120059">Angelus Bell of Sound Hada</a>,
 <a href="pedigree.asp?ID=1075734">Annabell of Cedar Springs (1978)</a>,
 <a href="pedigree.asp?ID=6135">Annabell Of Elsiville</a>,
 <a href="pedigree.asp?ID=611487">Annabell of Forest Gump (1995)</a>,
 <a href="pedigree.asp?ID=1036732">Annabell Of Shoot Flying Hill (1972)</a>,
 <a href="pedigree.asp?ID=487419">Augusta Bell of Thundering Heart (2011)</a>,
 <a 

```python
find_element(By.ID, "id")
find_element(By.NAME, "name")
find_element(By.XPATH, "xpath")
find_element(By.LINK_TEXT, "link text")
find_element(By.PARTIAL_LINK_TEXT, "partial link text")
find_element(By.TAG_NAME, "tag name")
find_element(By.CLASS_NAME, "class name")
find_element(By.CSS_SELECTOR, "css selector")
```